In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from tqdm import tqdm
from sklearn.decomposition import PCA
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F_torch


In [ ]:
train_test_split = 0.8
np.random.seed(0)

In [ ]:
data = np.genfromtxt('ml-32m/ratings.csv', delimiter=',', skip_header=1)

u_idx = {}
i_idx = {}
idx_u = []
idx_i = []
data_by_user_train = []
data_by_item_train = []

data_by_user_test = []
data_by_item_test = []

for i in range(data.shape[0]):
    user = int(data[i, 0])
    item = int(data[i, 1])
    rating = data[i, 2]

    if user not in u_idx:
        u_idx[user] = len(u_idx)
        idx_u.append(user)
        data_by_user_train.append([])
        data_by_user_test.append([])

    if item not in i_idx:
        i_idx[item] = len(i_idx)
        idx_i.append(item)
        data_by_item_train.append([])
        data_by_item_test.append([])
    
    u = (i_idx[item],rating)
    i = (u_idx[user],rating)

    if np.random.rand() < train_test_split:
        data_by_user_train[u_idx[user]].append(u)
        data_by_item_train[i_idx[item]].append(i)
    else:
        data_by_user_test[u_idx[user]].append(u)
        data_by_item_test[i_idx[item]].append(i)


In [ ]:
def vectorize(dataset, x):
    all_ratings = []
    ptr = [0]
    for ratings in dataset:
        all_ratings.extend(ratings)
        ptr.append(len(all_ratings))
   
    all_ratings = np.array(all_ratings, dtype=[(x, int), ('rating', float)])
    ptr = np.array(ptr, dtype=int)
    return all_ratings, ptr


In [ ]:
user_train , user_train_ptr = vectorize(data_by_user_train, x='item')
user_test , user_test_ptr = vectorize(data_by_user_test, x='item')

item_train, item_train_ptr = vectorize(data_by_item_train, x='user')
item_test, item_test_ptr = vectorize(data_by_item_test, x='user')


N = len(item_train_ptr) - 1
num_ratings = np.zeros(N, dtype=int)
for n in range(N):
    num_ratings[n] = (item_train_ptr[n+1] - item_train_ptr[n]) + (item_test_ptr[n+1] - item_test_ptr[n])

In [ ]:
movie_id_to_genres = {}
movie_titles = {}
all_genres = set()

with open("ml-32m/movies.csv", encoding="utf-8", newline="") as f:
    reader = csv.reader(f)
    next(reader)
    for movie_id_str, title, genres_str in reader:
        movie_id = int(movie_id_str)
        if genres_str == "(no genres listed)" :
            genres = []  
        else:
            genres = genres_str.split("|")

        movie_id_to_genres[movie_id] = genres
    
        movie_titles[movie_id] = title  
        all_genres.update(genres)

all_genres = sorted(all_genres)
genre_to_index = {g: i for i, g in enumerate(all_genres)}
num_features = len(all_genres)

N = len(idx_i)   
item_features = np.zeros((N, num_features), dtype=np.int8)

for n, movie_id in enumerate(idx_i):
    genres = movie_id_to_genres.get(movie_id, [])
    for g in genres:
        if g in genre_to_index:
            item_features[n, genre_to_index[g]] = 1


In [ ]:
ratings_train = user_train['rating']

plt.figure(figsize=(8, 5))
plt.hist(ratings_train, bins=np.arange(min(ratings_train), max(ratings_train) + 0.5, 0.5), edgecolor='black')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Histogram of user ratings (Train Set)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Rating distribution histogram.pdf", format="pdf")
plt.show()


In [ ]:
u_degrees = user_train_ptr[1:] - user_train_ptr[:-1] 
i_degrees = item_train_ptr[1:] - item_train_ptr[:-1]  

u_freq = Counter(u_degrees)
i_freq = Counter(i_degrees)

u_deg, u_counts = zip(*sorted(u_freq.items()))
i_deg, i_counts = zip(*sorted(i_freq.items()))

plt.figure(figsize=(8, 6))
plt.loglog(i_deg, i_counts, 'go', label='items', alpha=0.6)
plt.loglog(u_deg, u_counts, 'bs', label='users', alpha=0.6)

plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('User and item degree distributions')
plt.legend()
plt.grid(True, which="both", ls="--", lw=0.5)
plt.savefig("Power law distribution.pdf", format="pdf") 
plt.show()


In [ ]:
K=10
lam = 0.05
tau = 0.1
gamma = 0.1

M = len(u_idx)
N = len(i_idx)
max_iters = 50
tol = 1e-5

In [ ]:
#Bias only
def ALS_Bias(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, user_test_ptr, item_test_ptr, M, N, lam=1, gamma=0.1,max_iters=100, tol=1e-4, log=True):
    train_loss_history = []
    test_loss_history = []
    train_RMSE_history = []
    test_RMSE_history = []
 

    user_biases = np.zeros((M))
    item_biases = np.zeros((N))

    for iteration in range(max_iters):
        for m in range(M):
            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]
            item_ids = user_slice['item']
            ratings = user_slice['rating']

            if len(item_ids) == 0:
                continue
            
            i_bias = item_biases[item_ids]

            bias_residuals = ratings - i_bias

            denom = lam * len(item_ids) + gamma

            if denom == 0:
                user_biases[m] = 0
            else:
                user_biases[m] = lam * np.sum(bias_residuals) / denom


        for n in range(N):
            start = item_train_ptr[n]
            end = item_train_ptr[n+1]
            item_slice = item_train[start:end]
            user_ids = item_slice['user']
            ratings = item_slice['rating']

            if len(user_ids) == 0:
                continue

            u_bias = user_biases[user_ids]  
            bias_residuals = ratings - u_bias

            denom = lam * len(user_ids) + gamma
            if denom == 0:
                item_biases[n] = 0
            else:
                item_biases[n] = lam * np.sum(bias_residuals) / denom


        train_err =  0
        train_count = 0
        
        test_err = 0
        test_count = 0

        for m in range(M):
            u_bias = user_biases[m]

            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]

            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = u_bias + item_biases[item_ids]
                train_err += np.sum((ratings - preds) ** 2)
                train_count += len(ratings)
        
            start = user_test_ptr[m]
            end = user_test_ptr[m+1]
            user_slice = user_test[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = u_bias + item_biases[item_ids]
                test_err += np.sum((ratings - preds) ** 2)
                test_count += len(ratings)

        
        train_loss = 0.5 * lam  * train_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) 
        train_loss_history.append(train_loss)

        train_RMSE = np.sqrt(train_err / train_count)
        train_RMSE_history.append(train_RMSE)

        test_RMSE = np.sqrt(test_err / test_count)
        test_RMSE_history.append(test_RMSE)

        test_loss = 0.5 * lam  * test_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) 
        test_loss_history.append(test_loss)

        if log:
            print(f"Iter {iteration+1:3d} | " f"Train RMSE: {train_RMSE:.4f} | " f"Test RMSE: {test_RMSE:.4f} | " f"Train Loss: {train_loss:.2f} | " f"Test Loss: {test_loss:.2f}")

        if iteration > 4 and abs(test_RMSE_history[-1] - test_RMSE_history[-5]) < tol:
            if log:
                print(f"Early stopping at iteration {iteration + 1}")
            break

    history = {"train_loss": train_loss_history,"test_loss": test_loss_history, "train_RMSE": train_RMSE_history, "test_RMSE": test_RMSE_history}

    return user_biases, item_biases, history



In [ ]:
u_biases, i_biases, history =   ALS_Bias(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, 
                                user_test_ptr, item_test_ptr, M, N, lam=lam, gamma=gamma, max_iters=max_iters, tol=tol, log=True)
    

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_loss"], marker='o')
plt.title("Loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TRAIN LOSS BIAS.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["test_loss"], marker='o')
plt.title("Loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TEST LOSS BIAS.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_RMSE"], marker='o')
plt.plot(history["test_RMSE"], marker='*')
plt.legend(['Train Loss', 'Test Loss'])
plt.title("RMSE over Iterations - ALS with bias")
plt.xlabel("Iteration")
plt.ylabel("RMSE")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS RMSE BIAS.pdf", format="pdf")
plt.show()

In [ ]:
def ALS_Embedding(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, user_test_ptr, item_test_ptr, M, N,K,lam=1, gamma=0.1, tau=1, max_iters=100, tol=1e-4, log=True):
    train_loss_history = []
    test_loss_history = []
    train_RMSE_history = []
    test_RMSE_history = []

    user_biases = np.zeros((M))
    item_biases = np.zeros((N))

    U = np.random.normal(0, 0.1, size=(M, K))
    V = np.random.normal(0, 0.1, size=(N, K))

    for iteration in range(max_iters):
        for m in range(M):
            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]
            item_ids = user_slice['item']
            ratings = user_slice['rating']

            if len(item_ids) == 0:
                continue
            
            v = V[item_ids]
            i_bias = item_biases[item_ids]

            bias_residuals = ratings - np.dot(U[m], v.T) - i_bias

            denom = lam * len(item_ids) + gamma
            if denom == 0:
                user_biases[m] = 0
            else:
                user_biases[m] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - i_bias - user_biases[m]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * v, axis=0)
            A = lam * np.dot(v.T, v) + tau * np.eye(K)
            
            U[m] = np.linalg.solve(A, residual)


        for n in range(N):
            start = item_train_ptr[n]
            end = item_train_ptr[n+1]
            item_slice = item_train[start:end]
            user_ids = item_slice['user']
            ratings = item_slice['rating']

            if len(user_ids) == 0:
                continue

            u = U[user_ids]
            u_bias = user_biases[user_ids]

            bias_residuals = ratings - np.dot(u, V[n]) - u_bias

            denom = lam * len(user_ids) + gamma
            if denom == 0:
                item_biases[n] = 0
            else:
                item_biases[n] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - u_bias - item_biases[n]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * u, axis=0)
            A = lam * np.dot(u.T, u) + tau * np.eye(K)

            V[n] = np.linalg.solve(A, residual)


        train_err =  0
        train_count = 0
        
        test_err = 0
        test_count = 0

        for m in range(M):
            u_bias = user_biases[m]
            u_vec = U[m]

            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = np.dot(u_vec, V[item_ids].T) + u_bias + item_biases[item_ids]
                train_err += np.sum((ratings - preds) ** 2)
                train_count += len(ratings)
        
            start = user_test_ptr[m]
            end = user_test_ptr[m+1]
            user_slice = user_test[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = np.dot(u_vec, V[item_ids].T) + u_bias + item_biases[item_ids]
                test_err += np.sum((ratings - preds) ** 2)
                test_count += len(ratings)

        
        train_loss = 0.5 * lam  * train_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) + 0.5 * tau * (np.sum(U**2) + np.sum(V**2)) 
        train_loss_history.append(train_loss)

        train_RMSE = np.sqrt(train_err / train_count)
        train_RMSE_history.append(train_RMSE)

        test_RMSE = np.sqrt(test_err / test_count)
        test_RMSE_history.append(test_RMSE)

        test_loss = 0.5 * lam  * test_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) + 0.5 * tau * (np.sum(U**2) + np.sum(V**2)) 
        test_loss_history.append(test_loss)

        if log:
            print(f"Iter {iteration+1:3d} | " f"Train RMSE: {train_RMSE:.4f} | " f"Test RMSE: {test_RMSE:.4f} | " f"Train Loss: {train_loss:.2f} | " f"Test Loss: {test_loss:.2f}")

        if iteration > 4 and (abs(test_RMSE_history[-1] - test_RMSE_history[-5]) < tol or test_RMSE_history[-1] > test_RMSE_history[-2]):
            if log:
                print(f"Early stopping at iteration {iteration + 1}")
            break
    
    history = {"train_loss": train_loss_history,"test_loss": test_loss_history, "train_RMSE": train_RMSE_history, "test_RMSE": test_RMSE_history}

    return user_biases, item_biases, U, V ,history



In [ ]:
u_biases, i_biases, U , V, history =  ALS_Embedding(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, 
                                user_test_ptr, item_test_ptr, M, N, K, lam=lam, gamma=gamma, tau = tau, max_iters=max_iters, tol=tol, log=True)
    

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_loss"], marker='o')
plt.title("Train loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TRAIN LOSS EMBEDDING.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["test_loss"], marker='o')
plt.title("Test loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TEST LOSS EMBEDDING.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_RMSE"], marker='o')
plt.plot(history["test_RMSE"], marker='*')
plt.legend(['Train Loss', 'Test Loss'])
plt.title("RMSE over Iterations - ALS with bias and embedding")
plt.xlabel("Iteration")
plt.ylabel("RMSE")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS RMSE EMBEDDING.pdf", format="pdf")
plt.show()

In [ ]:
pca = PCA(n_components=2)
V_2d = pca.fit_transform(V)

Horror = [39446, 48877, 55577]
SciFi = [260, 1196, 1210]
RomCom = [3, 7, 64]
Children = [78499, 49274, 62999]

groups = { "Horror": Horror, "Sci-Fi": SciFi, "RomCom": RomCom, "Children": Children}
colors = {"Horror": "tab:orange", "Sci-Fi": "tab:blue", "RomCom": "tab:red","Children": "tab:purple"}

plt.figure(figsize=(10, 8))
plt.scatter(V_2d[:, 0], V_2d[:, 1], s=3, c="#00cc44", alpha=0.2, label="All movies")

for group_name, movie_ids in groups.items():
    indices = [idx_i.index(mid) for mid in movie_ids if mid in idx_i]
    plt.scatter(V_2d[indices, 0], V_2d[indices, 1], s=50, c=colors[group_name], label=group_name,
                edgecolors="black", linewidths=0.5)
    for idx in indices:
        movie_id = idx_i[idx]
        title = movie_titles.get(movie_id, f"movieId={movie_id}")
        plt.text(V_2d[idx, 0] + 0.02, V_2d[idx, 1] + 0.02, title, fontsize=8)


plt.axhline(0, lw=0.8, color="gray", alpha=0.5)
plt.axvline(0, lw=0.8, color="gray", alpha=0.5)
plt.title("2D Embedding of Movies")
plt.legend()
plt.tight_layout()
plt.savefig("2D embeddings.pdf", format="pdf")
plt.show()


In [ ]:
#vectorized with features
def ALS_Features(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, user_test_ptr, item_test_ptr, M, N,K, item_features , lam=1, gamma=0.1, tau=1, max_iters=100, tol=1e-4, log=True):
    train_loss_history = []
    test_loss_history = []
    train_RMSE_history = []
    test_RMSE_history = []

    user_biases = np.zeros((M))
    item_biases = np.zeros((N))
    num_features = item_features.shape[1]

    U = np.random.normal(0, 0.1, size=(M, K))
    V = np.random.normal(0, 0.1, size=(N, K))
    F = np.random.normal(0, 0.1, size=(num_features, K))

    for iteration in range(max_iters):
        for m in range(M):
            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]
            item_ids = user_slice['item']
            ratings = user_slice['rating']

            if len(item_ids) == 0:
                continue
            
            v = V[item_ids]
            i_bias = item_biases[item_ids]

            bias_residuals = ratings - np.dot(U[m], v.T) - i_bias

            denom = lam * len(item_ids) + gamma
            if denom == 0:
                user_biases[m] = 0
            else :
                user_biases[m] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - i_bias - user_biases[m]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * v, axis=0)
            A = lam * np.dot(v.T, v) + tau * np.eye(K)
            U[m] = np.linalg.solve(A, residual)

        for n in range(N):
            start = item_train_ptr[n]
            end = item_train_ptr[n+1]
            item_slice = item_train[start:end]
            user_ids = item_slice['user']
            ratings = item_slice['rating']

            if len(user_ids) == 0:
                continue

            u = U[user_ids]
            u_bias = user_biases[user_ids]

            bias_residuals = ratings - np.dot(u, V[n]) - u_bias
            denom = lam * len(user_ids) + gamma
            if denom == 0:
                item_biases[n] = 0 
            else:
                item_biases[n] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - u_bias - item_biases[n]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * u, axis=0)
            A = lam * np.dot(u.T, u) + tau * np.eye(K)

            f_idx = np.flatnonzero(item_features[n])  
            Fn = len(f_idx)
            if Fn > 0:
                feature_mean = F[f_idx].sum(axis=0) / np.sqrt(Fn)
            else:
                feature_mean = np.zeros(K)
            V[n] = np.linalg.solve(A, residual + tau * feature_mean)

        for l in range(num_features):
            items_with_l = np.flatnonzero(item_features[:, l])
            if items_with_l.size == 0:
                continue

            num = np.zeros(K)
            denom = 1
            for n in items_with_l:
                f_idx = np.flatnonzero(item_features[n])
                Fn = len(f_idx)
                if Fn == 0:
                    continue

                sum_except_l = F[f_idx].sum(axis=0) - F[l]
                num += (V[n] - (sum_except_l / np.sqrt(Fn))) / np.sqrt(Fn)   
                denom += 1 / Fn
            F[l] = num / denom


        train_err =  0
        train_count = 0
        test_err = 0
        test_count = 0

        for m in range(M):
            u_bias = user_biases[m]
            u_vec = U[m]
            start = user_train_ptr[m]
            end = user_train_ptr[m+1]
            user_slice = user_train[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = np.dot(u_vec, V[item_ids].T) + u_bias + item_biases[item_ids]
                train_err += np.sum((ratings - preds) ** 2)
                train_count += len(ratings)
            start = user_test_ptr[m]
            end = user_test_ptr[m+1]
            user_slice = user_test[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = np.dot(u_vec, V[item_ids].T) + u_bias + item_biases[item_ids]
                test_err += np.sum((ratings - preds) ** 2)
                test_count += len(ratings)
        
        feature_prior = 0
        for n in range(N):
            f_idx = np.flatnonzero(item_features[n])
            Fn = len(f_idx)
            if Fn > 0:
                feature_mean = F[f_idx].sum(axis=0) / np.sqrt(Fn)
            else:
                feature_mean = np.zeros(K)
            feature_prior += np.sum((V[n] - feature_mean) ** 2)

        train_loss = 0.5 * lam  * train_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) + 0.5 * tau * (np.sum(U**2) +feature_prior + np.sum(F**2)) 
        train_loss_history.append(train_loss)

        train_RMSE = np.sqrt(train_err / train_count)
        train_RMSE_history.append(train_RMSE)

        test_RMSE = np.sqrt(test_err / test_count)
        test_RMSE_history.append(test_RMSE)

        test_loss = 0.5 * lam  * test_err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) + 0.5 * tau * (np.sum(U**2) +feature_prior + np.sum(F**2)) 
        test_loss_history.append(test_loss)

        if log:
            print(f"Iter {iteration+1:3d} | " f"Train RMSE: {train_RMSE:.4f} | " f"Test RMSE: {test_RMSE:.4f} | " f"Train Loss: {train_loss:.2f} | " f"Test Loss: {test_loss:.2f}")

        if iteration > 4 and abs(test_RMSE_history[-1] - test_RMSE_history[-5]) < tol:
            if log:
                print(f"Early stopping at iteration {iteration + 1}")
            break

    history = {"train_loss": train_loss_history,"test_loss": test_loss_history, "train_RMSE": train_RMSE_history, "test_RMSE": test_RMSE_history}

    return user_biases, item_biases, U, V , F, history


In [ ]:
u_biases, i_biases, U , V, F, history =  ALS_Features(user_train, item_train, user_test, item_test, user_train_ptr, item_train_ptr, 
                                user_test_ptr, item_test_ptr, M, N, K, item_features,lam=lam, gamma=gamma, tau = tau, max_iters=max_iters, tol=tol, log=True)
    

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_loss"], marker='o')
plt.title("Loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TRAIN LOSS FEATURES.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["test_loss"], marker='o')
plt.title("Loss over Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS TEST LOSS FEATURES.pdf", format="pdf")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history["train_RMSE"], marker='o')
plt.plot(history["test_RMSE"], marker='*')
plt.legend(['Train Loss', 'Test Loss'])
plt.title("RMSE over Iterations - ALS with bias, embedding and features")
plt.xlabel("Iteration")
plt.ylabel("RMSE")
plt.grid(True)
plt.tight_layout()
plt.savefig("ALS RMSE FEATURES.pdf", format="pdf")
plt.show()

In [ ]:
pca = PCA(n_components=2)
V_2d = pca.fit_transform(V)

Horror = [39446, 48877, 55577]
SciFi = [260, 1196, 1210]
RomCom = [3, 7, 64]
Children = [78499, 49274, 62999]

groups = { "Horror": Horror, "Sci-Fi": SciFi, "RomCom": RomCom, "Children": Children}
colors = {"Horror": "tab:orange", "Sci-Fi": "tab:blue", "RomCom": "tab:red","Children": "tab:purple"}

plt.figure(figsize=(10, 8))
plt.scatter(V_2d[:, 0], V_2d[:, 1], s=3, c="#00cc44", alpha=0.2, label="All movies")

for group_name, movie_ids in groups.items():
    indices = [idx_i.index(mid) for mid in movie_ids if mid in idx_i]
    plt.scatter(V_2d[indices, 0], V_2d[indices, 1], s=50, c=colors[group_name], label=group_name,
                edgecolors="black", linewidths=0.5)
    for idx in indices:
        movie_id = idx_i[idx]
        title = movie_titles.get(movie_id, f"movieId={movie_id}")
        plt.text(V_2d[idx, 0] + 0.02, V_2d[idx, 1] + 0.02, title, fontsize=8)


plt.axhline(0, lw=0.8, color="gray", alpha=0.5)
plt.axvline(0, lw=0.8, color="gray", alpha=0.5)
plt.title("2D Embedding of Movies")
plt.legend()
plt.tight_layout()
plt.savefig("2D embeddings with features.pdf", format="pdf")
plt.show()


In [ ]:
pca = PCA(n_components=2, random_state=0)
V_2d = pca.fit_transform(V)         
F_2d = pca.transform(F)              

index_to_genre = {i: g for g, i in genre_to_index.items()}

genres_to_annotate = ["Action", "Comedy", "Drama", "Sci-Fi", "Romance"]
annot_idx = [genre_to_index[g] for g in genres_to_annotate if g in genre_to_index]

plt.figure(figsize=(11, 8.5))

plt.scatter(V_2d[:, 0], V_2d[:, 1], s=3, c="#00cc44", alpha=0.18, label="All movies")

plt.scatter(F_2d[:, 0], F_2d[:, 1], s=120, marker="*", edgecolors="black",
            linewidths=0.6, alpha=0.95, c="tab:purple", label="Feature vectors (genres)")

for gi in range(F_2d.shape[0]):
    x, y = F_2d[gi]
    plt.text(x + 0.02, y + 0.02, index_to_genre[gi], fontsize=8)


plt.axhline(0, lw=0.8, color="gray", alpha=0.45)
plt.axvline(0, lw=0.8, color="gray", alpha=0.45)
plt.title("2D PCA of Movie Embeddings with Features", fontsize=12)
plt.legend(loc="best", fontsize=9, frameon=True)
plt.tight_layout()
plt.savefig("FEATURE EMBEDDINGS.pdf", format="pdf")
plt.show()


In [ ]:
# Cars 45517  LOTR 2116, 4993   Cold start movie 52696

#Horror = [39446, 48877, 55577]
#SciFi = [260, 1196, 1210]
#RomCom = [3, 7, 64]
#Superhero = [33794, 59315, 77561]

movieID = 79363

r_dummy = 5
u_dummy = np.zeros(K)
b_dummy = 0

is_cold_start = False

if movieID in i_idx:
    item_idx = i_idx[movieID]
    v = V[item_idx]
    i_bias = i_biases[item_idx]
else:
    is_cold_start = True
    genres = movie_id_to_genres.get(movieID, [])
    f_idx = [genre_to_index[g] for g in genres if g in genre_to_index]

    if not f_idx:
        v = np.random.normal(0, 0.01, size=K)
        i_bias = 0
    else:
        Fn = len(f_idx)
        v = F[f_idx].sum(axis=0) / np.sqrt(Fn)  
        i_bias = np.mean(i_biases)                    

for i in range(20):
    b_dummy = lam * (r_dummy - np.dot(u_dummy, v) - i_bias) / (lam + gamma)
    u_dummy = ((r_dummy - b_dummy - i_bias) * v) / (np.dot(v, v) + tau)

scores = V @ u_dummy + 0.05 * i_biases

min_ratings = 100
scores = np.where(np.array(num_ratings) >= min_ratings, scores, -np.inf)

top_items = np.argsort(scores)[::-1]
top_n = 5
shown = 0

print(f"\nTop {top_n} recommendations for a dummy " f"'{movie_titles.get(movieID, f'movieId={movieID}')}' fan ")

for idx in top_items:
    movie_id = idx_i[idx]
    if movie_id == movieID:
        continue
    title = movie_titles.get(movie_id, f"movieId={movie_id}")
    genres = movie_id_to_genres.get(movie_id, ["(no genres listed)"])

    print(f"{shown+1:2d}. {title} — Genres: {', '.join(genres)}")
    shown += 1
    if shown >= top_n:
        break


In [ ]:
movie_norms = np.linalg.norm(V, axis=1)

min_ratings = 100
valid_indices = [i for i, n in enumerate(num_ratings) if n >= min_ratings]

polarizing_indices = sorted(valid_indices, key=lambda i: movie_norms[i], reverse=True)
least_polarizing_indices = sorted(valid_indices, key=lambda i: movie_norms[i])

top_n = 10

print(f"\nTop {top_n} MOST polarizing movies:")
for idx in polarizing_indices[:top_n]:
    movie_id = idx_i[idx]
    print(f"- {movie_titles.get(movie_id, f'movieId={movie_id}')} " f"(norm={movie_norms[idx]:.4f}, ratings={num_ratings[idx]})")

print(f"\nTop {top_n} LEAST polarizing movies:")
for idx in least_polarizing_indices[:top_n]:
    movie_id = idx_i[idx]
    print(f"- {movie_titles.get(movie_id, f'movieId={movie_id}')} " f"(norm={movie_norms[idx]:.4f}, ratings={num_ratings[idx]})")


In [ ]:
data = np.genfromtxt('ml-32m/ratings.csv', delimiter=',', skip_header=1)

u_idx = {}
i_idx = {}
idx_u = []
idx_i = []

data_by_user = []
data_by_item = []

for i in range(data.shape[0]):
    user = int(data[i, 0])
    item = int(data[i, 1])
    rating = data[i, 2]

    if user not in u_idx:
        u_idx[user] = len(u_idx)
        idx_u.append(user)
        data_by_user.append([])

    if item not in i_idx:
        i_idx[item] = len(i_idx)
        idx_i.append(item)
        data_by_item.append([])
    
    u = (i_idx[item],rating)
    i = (u_idx[user],rating)

 
    data_by_user[u_idx[user]].append(u)
    data_by_item[i_idx[item]].append(i)


user_full , user_full_ptr = vectorize(data_by_user, x='item')
item_full, item_full_ptr = vectorize(data_by_item, x='user')

N = len(item_full) - 1
num_ratings = np.diff(item_full_ptr) 



In [ ]:
#vectorized with features full set
def ALS_Features_Full(user_full, item_full, user_full_ptr, item_full_ptr, M, N,K, item_features , lam=1, gamma=0.1, tau=1, max_iters=100, tol=1e-4, log=True):
    loss_history = []
    RMSE_history = []

    user_biases = np.zeros((M))
    item_biases = np.zeros((N))
    num_features = item_features.shape[1]

    U = np.random.normal(0, 0.1, size=(M, K))
    V = np.random.normal(0, 0.1, size=(N, K))
    F = np.random.normal(0, 0.1, size=(num_features, K))

    for iteration in range(max_iters):
        for m in range(M):
            start = user_full_ptr[m]
            end = user_full_ptr[m+1]
            user_slice = user_full[start:end]
            item_ids = user_slice['item']
            ratings = user_slice['rating']

            if len(item_ids) == 0:
                continue
            
            v = V[item_ids]
            i_bias = item_biases[item_ids]

            bias_residuals = ratings - np.dot(U[m], v.T) - i_bias

            denom = lam * len(item_ids) + gamma
            if denom == 0:
                user_biases[m] = 0
            else :
                user_biases[m] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - i_bias - user_biases[m]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * v, axis=0)
            A = lam * np.dot(v.T, v) + tau * np.eye(K)
            U[m] = np.linalg.solve(A, residual)

        for n in range(N):
            start = item_full_ptr[n]
            end = item_full_ptr[n+1]
            item_slice = item_full[start:end]
            user_ids = item_slice['user']
            ratings = item_slice['rating']

            if len(user_ids) == 0:
                continue

            u = U[user_ids]
            u_bias = user_biases[user_ids]

            bias_residuals = ratings - np.dot(u, V[n]) - u_bias
            denom = lam * len(user_ids) + gamma
            if denom == 0:
                item_biases[n] = 0 
            else:
                item_biases[n] = lam * np.sum(bias_residuals) / denom

            vector_residual = ratings - u_bias - item_biases[n]
            residual = lam * np.sum(vector_residual[:, np.newaxis] * u, axis=0)
            A = lam * np.dot(u.T, u) + tau * np.eye(K)

            f_idx = np.flatnonzero(item_features[n])  
            Fn = len(f_idx)
            if Fn > 0:
                feature_mean = F[f_idx].sum(axis=0) / np.sqrt(Fn)
            else:
                feature_mean = np.zeros(K)
            V[n] = np.linalg.solve(A, residual + tau * feature_mean)

        for l in range(num_features):
            items_with_l = np.flatnonzero(item_features[:, l])
            if items_with_l.size == 0:
                continue

            num = np.zeros(K)
            denom = 1
            for n in items_with_l:
                f_idx = np.flatnonzero(item_features[n])
                Fn = len(f_idx)
                if Fn == 0:
                    continue

                sum_except_l = F[f_idx].sum(axis=0) - F[l]
                num += (V[n] - (sum_except_l / np.sqrt(Fn))) / np.sqrt(Fn)   
                denom += 1 / Fn
            F[l] = num / denom


        err =  0
        count = 0

        for m in range(M):
            u_bias = user_biases[m]
            u_vec = U[m]
            start = user_full_ptr[m]
            end = user_full_ptr[m+1]
            user_slice = user_full[start:end]
            if len(user_slice) > 0:
                item_ids = user_slice['item']
                ratings = user_slice['rating']
                preds = np.dot(u_vec, V[item_ids].T) + u_bias + item_biases[item_ids]
                err += np.sum((ratings - preds) ** 2)
                count += len(ratings)
        
        feature_prior = 0
        for n in range(N):
            f_idx = np.flatnonzero(item_features[n])
            Fn = len(f_idx)
            if Fn > 0:
                feature_mean = F[f_idx].sum(axis=0) / np.sqrt(Fn)
            else:
                feature_mean = np.zeros(K)
            feature_prior += np.sum((V[n] - feature_mean) ** 2)

        loss = 0.5 * lam  * err + 0.5 * gamma * (np.sum(user_biases ** 2) + np.sum(item_biases ** 2)) + 0.5 * tau * (np.sum(U**2) +feature_prior + np.sum(F**2)) 
        loss_history.append(loss)

        RMSE = np.sqrt(err / count)
        RMSE_history.append(RMSE)


        if log:
            print(f"Iter {iteration+1:3d}  | " f" Loss: {loss:.4f} | " f" RMSE: {RMSE:.4f}")

        if iteration > 4 and abs(RMSE_history[-1] - RMSE_history[-5]) < tol:
            if log:
                print(f"Early stopping at iteration {iteration + 1}")
            break

    history = {"loss": loss_history, "RMSE": RMSE_history}

    return user_biases, item_biases, U, V , F, history


In [ ]:
u_biases, i_biases, U , V, F, history =  ALS_Features_Full(user_full, item_full, user_full_ptr, item_full_ptr, M, N, K, item_features,
                                                    lam=lam, gamma=gamma, tau = tau, max_iters=50, tol=1e-4, log=True)
    

In [ ]:
pca = PCA(n_components=2)
V_2d = pca.fit_transform(V)

Horror = [39446, 48877, 55577]
SciFi = [260, 1196, 1210]
RomCom = [3, 7, 64]
Children = [78499, 49274, 62999]

groups = { "Horror": Horror, "Sci-Fi": SciFi, "RomCom": RomCom, "Children": Children}
colors = {"Horror": "tab:orange", "Sci-Fi": "tab:blue", "RomCom": "tab:red","Children": "tab:purple"}

plt.figure(figsize=(10, 8))
plt.scatter(V_2d[:, 0], V_2d[:, 1], s=3, c="#00cc44", alpha=0.2, label="All movies")

for group_name, movie_ids in groups.items():
    indices = [idx_i.index(mid) for mid in movie_ids if mid in idx_i]
    plt.scatter(V_2d[indices, 0], V_2d[indices, 1], s=50, c=colors[group_name], label=group_name,
                edgecolors="black", linewidths=0.5)
    for idx in indices:
        movie_id = idx_i[idx]
        title = movie_titles.get(movie_id, f"movieId={movie_id}")
        plt.text(V_2d[idx, 0] + 0.02, V_2d[idx, 1] + 0.02, title, fontsize=8)


plt.axhline(0, lw=0.8, color="gray", alpha=0.5)
plt.axvline(0, lw=0.8, color="gray", alpha=0.5)
plt.title("2D Embedding of Movies")
plt.legend()
plt.tight_layout()
plt.savefig("2D embeddings with features full set.pdf", format="pdf")
plt.show()


In [ ]:
pca = PCA(n_components=2, random_state=0)
V_2d = pca.fit_transform(V)         
F_2d = pca.transform(F)              

index_to_genre = {i: g for g, i in genre_to_index.items()}

genres_to_annotate = ["Action", "Comedy", "Drama", "Sci-Fi", "Romance"]
annot_idx = [genre_to_index[g] for g in genres_to_annotate if g in genre_to_index]

plt.figure(figsize=(11, 8.5))

plt.scatter(V_2d[:, 0], V_2d[:, 1], s=3, c="#00cc44", alpha=0.18, label="All movies")

plt.scatter(F_2d[:, 0], F_2d[:, 1], s=120, marker="*", edgecolors="black",
            linewidths=0.6, alpha=0.95, c="tab:purple", label="Feature vectors (genres)")

for gi in range(F_2d.shape[0]):
    x, y = F_2d[gi]
    plt.text(x + 0.02, y + 0.02, index_to_genre[gi], fontsize=8)


plt.axhline(0, lw=0.8, color="gray", alpha=0.45)
plt.axvline(0, lw=0.8, color="gray", alpha=0.45)
plt.title("2D PCA of Movie Embeddings with Features", fontsize=12)
plt.legend(loc="best", fontsize=9, frameon=True)
plt.tight_layout()
plt.savefig("FEATURE EMBEDDINGS full set.pdf", format="pdf")
plt.show()


In [ ]:
# Cars 45517  LOTR 2116, 4993   Cold start movie 52696

#Horror = [39446, 48877, 55577]
#SciFi = [260, 1196, 1210]
#RomCom = [3, 7, 64]
#Superhero = [33794, 59315, 77561]

movieID = 79363

r_dummy = 5
u_dummy = np.zeros(K)
b_dummy = 0

is_cold_start = False

if movieID in i_idx:
    item_idx = i_idx[movieID]
    v = V[item_idx]
    i_bias = i_biases[item_idx]
else:
    is_cold_start = True
    genres = movie_id_to_genres.get(movieID, [])
    f_idx = [genre_to_index[g] for g in genres if g in genre_to_index]

    if not f_idx:
        v = np.random.normal(0, 0.01, size=K)
        i_bias = 0
    else:
        Fn = len(f_idx)
        v = F[f_idx].sum(axis=0) / np.sqrt(Fn)  
        i_bias = np.mean(i_biases)                    

for i in range(20):
    b_dummy = lam * (r_dummy - np.dot(u_dummy, v) - i_bias) / (lam + gamma)
    u_dummy = ((r_dummy - b_dummy - i_bias) * v) / (np.dot(v, v) + tau)

scores = V @ u_dummy + 0.05 * i_biases

min_ratings = 100
scores = np.where(np.array(num_ratings) >= min_ratings, scores, -np.inf)

top_items = np.argsort(scores)[::-1]
top_n = 5
shown = 0

print(f"\nTop {top_n} recommendations for a dummy " f"'{movie_titles.get(movieID, f'movieId={movieID}')}' fan ")

for idx in top_items:
    movie_id = idx_i[idx]
    if movie_id == movieID:
        continue
    title = movie_titles.get(movie_id, f"movieId={movie_id}")
    genres = movie_id_to_genres.get(movie_id, ["(no genres listed)"])

    print(f"{shown+1:2d}. {title} — Genres: {', '.join(genres)}")
    shown += 1
    if shown >= top_n:
        break


In [ ]:
movie_norms = np.linalg.norm(V, axis=1)

min_ratings = 100
valid_indices = [i for i, n in enumerate(num_ratings) if n >= min_ratings]

polarizing_indices = sorted(valid_indices, key=lambda i: movie_norms[i], reverse=True)
least_polarizing_indices = sorted(valid_indices, key=lambda i: movie_norms[i])

top_n = 10

print(f"\nTop {top_n} MOST polarizing movies:")
for idx in polarizing_indices[:top_n]:
    movie_id = idx_i[idx]
    print(f"- {movie_titles.get(movie_id, f'movieId={movie_id}')} " f"(norm={movie_norms[idx]:.4f}, ratings={num_ratings[idx]})")

print(f"\nTop {top_n} LEAST polarizing movies:")
for idx in least_polarizing_indices[:top_n]:
    movie_id = idx_i[idx]
    print(f"- {movie_titles.get(movie_id, f'movieId={movie_id}')} " f"(norm={movie_norms[idx]:.4f}, ratings={num_ratings[idx]})")


In [ ]:
def objective(trial):
    lam = trial.suggest_float("lam", 0.02, 0.2, log=True)
    gamma = trial.suggest_float("gamma", 0.05, 0.3, log=True)

    user_biases, item_biases, history = ALS_Bias(
        user_train=user_train,
        item_train=item_train,
        user_test=user_test,
        item_test=item_test,
        user_train_ptr=user_train_ptr,
        item_train_ptr=item_train_ptr,
        user_test_ptr=user_test_ptr,
        item_test_ptr=item_test_ptr,
        M=M,
        N=N,
        lam=lam,
        gamma=gamma,
        max_iters=max_iters,
        tol=tol,
        log=False      
    )

    test_rmse = history["test_RMSE"][-1]

    for i, rm in enumerate(history["test_RMSE"], start=1):
        trial.report(rm, i)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return test_rmse

sampler = TPESampler(seed=0, multivariate=True)
pruner = MedianPruner(n_startup_trials=10, n_warmup_steps=5)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Best value (test RMSE):", study.best_value)
print("Best params:", study.best_params)



In [ ]:
def objective(trial):
    K = trial.suggest_categorical("K", [5, 8,10, 12, 15 ])
    lam = trial.suggest_float("lam", 0.02, 0.2, log=True)
    gamma = trial.suggest_float("gamma", 0.05, 0.3, log=True)
    tau = trial.suggest_float("tau", 0.02, 0.2, log=True)



    user_biases, item_biases, U, V, history = ALS_Embedding(
        user_train=user_train,
        item_train=item_train,
        user_test=user_test,
        item_test=item_test,
        user_train_ptr=user_train_ptr,
        item_train_ptr=item_train_ptr,
        user_test_ptr=user_test_ptr,
        item_test_ptr=item_test_ptr,
        M=M,
        N=N,
        K=K,
        lam=lam,
        gamma=gamma,
        tau=tau,
        max_iters=max_iters,
        tol=tol,
        log=False)

    for step, rm in enumerate(history["test_RMSE"], start=1):
        trial.report(rm, step)
        if trial.should_prune():
            raise optuna.TrialPruned()


    return history["test_RMSE"][-1]


sampler = TPESampler(seed=0, multivariate=True)
pruner = MedianPruner(n_startup_trials=10, n_warmup_steps=5)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Best value (test RMSE):", study.best_value)
print("Best params:", study.best_params)


In [ ]:
def objective(trial):
    K = trial.suggest_categorical("K", [8 ,10, 12])
    lam = trial.suggest_float("lam", 0.01, 0.1, log=True)
    gamma = trial.suggest_float("gamma", 0.05, 0.15, log=True)
    tau = trial.suggest_float("tau", 0.05, 0.15, log=True)

    user_biases, item_biases, U, V, F, history = ALS_Features(
        user_train=user_train,
        item_train=item_train,
        user_test=user_test,
        item_test=item_test,
        user_train_ptr=user_train_ptr,
        item_train_ptr=item_train_ptr,
        user_test_ptr=user_test_ptr,
        item_test_ptr=item_test_ptr,
        M=M,
        N=N,
        K=K,
        item_features=item_features,
        lam=lam,
        gamma=gamma,
        tau=tau,
        max_iters= 50,
        tol=tol,
        log=False )

    for step, rm in enumerate(history["test_RMSE"], start=1):
        trial.report(rm, step)
        if trial.should_prune():
            raise optuna.TrialPruned()

   
    return history["test_RMSE"][-1]


sampler = TPESampler(seed=0, multivariate=True)
pruner = MedianPruner(n_startup_trials=10, n_warmup_steps=5)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=15, show_progress_bar=True)

print("Best value (test RMSE):", study.best_value)
print("Best params:", study.best_params)



In [ ]:
def build_implicit(data_by_user, data_by_item, threshold=4.0):
    M = len(data_by_user)
    positive_items_by_user = [set() for _ in range(M)]
    for m in range(M):
        for n, r in data_by_user[m]:
            if r >= threshold:
                positive_items_by_user[m].add(n)
    return positive_items_by_user

def sample(pos_items_by_user, N, batch_users, max_resample=20, device="cpu"):
    u = torch.tensor(batch_users, dtype=torch.long, device=device)
    i = torch.tensor([np.random.choice(list(pos_items_by_user[user])) for user in batch_users],dtype=torch.long, device=device)

    j_list = np.random.randint(0, N, size=len(batch_users)).tolist()
    for k, user in enumerate(batch_users):
        tries = 0
        while j_list[k] in pos_items_by_user[user] and tries < max_resample:
            j_list[k] = np.random.randint(0, N)
            tries += 1
    j = torch.tensor(j_list, dtype=torch.long, device=device)

    return u, i, j

class BPR(nn.Module):
    def __init__(self, M, N, K):
        super().__init__()
        self.user_emb = nn.Embedding(M, K)
        self.item_emb = nn.Embedding(N, K)
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, u, i, j):
        u_vec = self.user_emb(u)
        i_vec = self.item_emb(i)
        j_vec = self.item_emb(j)
        r_ui = torch.sum(u_vec * i_vec, dim=1)
        r_uj = torch.sum(u_vec * j_vec, dim=1)
        return r_ui - r_uj 

def train_bpr_torch(model, pos_items_by_user,n_epochs=10, batch_size=4096, lr=0.01, reg=0.01,seed=0, device="cpu"):
    torch.manual_seed(seed)
    rng = np.random.default_rng(seed)

    M = model.user_emb.num_embeddings
    N = model.item_emb.num_embeddings

    train_users = np.array([u for u in range(M) if len(pos_items_by_user[u]) > 0], dtype=np.int64)
    steps_per_epoch = max(1, len(train_users) // batch_size)

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, n_epochs+1):
        rng.shuffle(train_users)

        for step in range(steps_per_epoch):
            batch_users = train_users[step*batch_size:(step+1)*batch_size]
            if len(batch_users) == 0:
                continue

            u, i, j = sample(pos_items_by_user, N, batch_users, device=device)
            r_uij = model(u, i, j)

            loss = -F_torch.logsigmoid(r_uij).mean()
            reg_loss = sum(torch.norm(p, p=2).pow(2) for p in model.parameters())
            loss += reg * reg_loss / batch_size

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}/{n_epochs}, Loss={loss.item():.4f}")

    return model


def Evaluate_at_k(model, test_pos_by_user, train_pos_by_user=None, K=10):
    U = model.user_emb.weight.detach().cpu().numpy()
    V = model.item_emb.weight.detach().cpu().numpy()

    precisions, recalls, ndcgs = [], [], []

    for u, true_items in enumerate(test_pos_by_user):
        if not true_items:
            continue
        exclude = train_pos_by_user[u] if train_pos_by_user else set()

        scores = U[u] @ V.T
        if exclude:
            scores[list(exclude)] = -np.inf
        topk = np.argsort(-scores)[:K]

        hits = [1 if i in true_items else 0 for i in topk]
        n_hit = sum(hits)
        precisions.append(n_hit / K)
        recalls.append(n_hit / len(true_items))

        dcg = 0
        for idx, h in enumerate(hits):
            dcg += h / np.log2(idx + 2)

        idcg = 0
        for i in range(min(len(true_items), K)):
            idcg += 1 / np.log2(i + 2)

        if idcg > 0:
            ndcgs.append(dcg / idcg)
        else:
            ndcgs.append(0)

    return np.mean(precisions), np.mean(recalls), np.mean(ndcgs)


In [ ]:
M = len(data_by_user_train)   
N = len(data_by_item_train)   
K = 50                        

pos_items_by_user_train = build_implicit(data_by_user_train, data_by_item_train, threshold=4.0)
pos_items_by_user_test  = build_implicit(data_by_user_test,  data_by_item_test,  threshold=4.0)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BPR(M, N, K)
trained_model = train_bpr_torch(model, pos_items_by_user_train,n_epochs=20, batch_size=4096,lr=0.004, reg=0.002, device=device)


In [ ]:
P, R, NDCG = Evaluate_at_k(trained_model, pos_items_by_user_test, pos_items_by_user_train, K=50)
print(f"Precision={P:.4f}, Recall={R:.4f}, NDCG={NDCG:.4f}")

In [ ]:
def objective(trial):
    K          = trial.suggest_categorical("K", [32, 50, 100, 200])
    lr         = trial.suggest_float("lr", 1e-4, 5e-2, log=True)
    reg        = trial.suggest_float("reg", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [1024, 2048, 4096, 8192])
    n_epochs   = trial.suggest_int("n_epochs", 10, 30)
    seed       = trial.suggest_int("seed", 0, 10)


    M = len(data_by_user_train)
    N = len(data_by_item_train)
    model = BPR(M, N, K)

   
    device = "cuda" if torch.cuda.is_available() else "cpu"
    trained = train_bpr_torch(
        model,
        pos_items_by_user_train,
        n_epochs=n_epochs,
        batch_size=batch_size,
        lr=lr,
        reg=reg,
        seed=seed,
        device=device,
    )


    P, R, NDCG = Evaluate_at_k(
        trained,
        pos_items_by_user_test,
        pos_items_by_user_train,
        K=10
    )


    trial.set_user_attr("Precision@10", float(P))
    trial.set_user_attr("Recall@10",    float(R))


    return float(NDCG)


study = optuna.create_study(direction="maximize", study_name="BPR_implicit_tuning")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("\nBest trial:")
print(f"  Value (NDCG@10): {study.best_value:.6f}")
print("  Params:")
for k, v in study.best_params.items():
    print(f"    {k}: {v}")


best = study.best_params
K_best          = best["K"]
lr_best         = best["lr"]
reg_best        = best["reg"]
batch_best      = best["batch_size"]
epochs_best     = best["n_epochs"]
seed_best       = best["seed"]

M = len(data_by_user_train)
N = len(data_by_item_train)
final_model = BPR(M, N, K_best)

device = "cuda" if torch.cuda.is_available() else "cpu"
final_model = train_bpr_torch(
    final_model,
    pos_items_by_user_train,
    n_epochs=epochs_best,
    batch_size=batch_best,
    lr=lr_best,
    reg=reg_best,
    seed=seed_best,
    device=device,
)

P, R, NDCG = Evaluate_at_k(final_model, pos_items_by_user_test, pos_items_by_user_train, K=10)
print(f"\nFinal (best) — Precision@10={P:.6f}, Recall@10={R:.6f}, NDCG@10={NDCG:.6f}")
print("Best hyperparameters:", study.best_params)


In [ ]:
def createreco(userid, versionid, user_ratings):
    u_dummy = np.zeros(K)
    b_dummy = 0

    min_ratings = 100

    for (item_idx, r_dummy) in user_ratings:
        movieID = idx_i[item_idx]

        if movieID in i_idx:
            v = V[item_idx]
            i_bias = i_biases[item_idx]
        else:
            genres = movie_id_to_genres.get(movieID, [])
            f_idx = [genre_to_index[g] for g in genres if g in genre_to_index]
            if not f_idx:
                v = np.random.normal(0, 0.01, size=K)
                i_bias = 0
            else:
                Fn = len(f_idx)
                v = F[f_idx].sum(axis=0) / np.sqrt(Fn)
                i_bias = np.mean(i_biases)

        for _ in range(20):
            b_dummy = lam * (r_dummy - np.dot(u_dummy, v) - i_bias) / (lam + gamma)
            u_dummy = ((r_dummy - b_dummy - i_bias) * v) / (np.dot(v, v) + tau)

    bias_weight = 0.05 if versionid == "A" else 0.2
    scores = V @ u_dummy + bias_weight * i_biases
    scores = np.where(np.array(num_ratings) >= min_ratings, scores, -np.inf)

    top_n = 5
    top_items = np.argsort(scores)[::-1]
    recommendations = []
    shown = 0

    rated_items = set([i for i, _ in user_ratings]) 

    for idx in top_items:
        if idx in rated_items:
            continue
        movie_id = idx_i[idx]
        title = movie_titles.get(movie_id, f"movieId={movie_id}")
        recommendations.append((movie_id, title))
        shown += 1
        if shown >= top_n:
            break

    return recommendations


In [ ]:
rng = np.random.default_rng(0)
num_dummy_users = 100
movies_per_user = 50
num_items = len(idx_i)

high_rating = np.array([4.0, 4.5, 5.0])  
all_ratings = np.arange(1.0, 5.5, 0.5) 

genre_to_movies = {}
for movie_id, genres in movie_id_to_genres.items():
    for g in genres:
        if g not in genre_to_movies:
            genre_to_movies[g] = []
        if movie_id in i_idx:
            genre_to_movies[g].append(i_idx[movie_id]) 

dummy_dataset = []
preferred_movies_by_user = []
preferred_genres = []

for u in range(num_dummy_users):
    preferred_genre = rng.choice(all_genres)
    preferred_genres.append(preferred_genre)
    preferred_movies = rng.choice(genre_to_movies[preferred_genre], size=movies_per_user, replace=False)
    preferred_ratings = [(idx, rng.choice(high_rating)) for idx in preferred_movies]
    ratings = preferred_ratings
    rng.shuffle(ratings)
    dummy_dataset.append(ratings)
    preferred_movies_by_user.append(preferred_movies)

dummy_user_ratings, dummy_user_ptr = vectorize(dummy_dataset, "item")

version_ids = rng.choice(["A", "B"], size=num_dummy_users)
group_A_users = np.where(version_ids == "A")[0]
group_B_users = np.where(version_ids == "B")[0]


In [ ]:
logs = []
genre_hits_A, genre_hits_B = 0, 0
k = 5  

for u in range(num_dummy_users):
    version = version_ids[u]
    start = dummy_user_ptr[u]
    end = dummy_user_ptr[u + 1]
    all_user_ratings = dummy_user_ratings[start:end]  

    train_ratings = list(all_user_ratings)
    recs = createreco(userid=u, versionid=version, user_ratings=train_ratings)
    for rank, (movie_id, title) in enumerate(recs, start=1):
        logs.append({"user_id": u, "version": version,  "Rank": rank, "movie_id": movie_id, "title": title })


    preferred_genre = preferred_genres[u]
    rec_genres = [movie_id_to_genres.get(movie_id, []) for movie_id, _ in recs]
    hits = sum(1 for genres in rec_genres if preferred_genre in genres)


    if version == "A":
        genre_hits_A += hits
    else:
        genre_hits_B += hits


logs_df = pd.DataFrame(logs)
logs_df.to_csv("AB Testing recommendations.csv", index=False)
print("Saved recommendations for 100 dummy users to AB Testing recommendations.csv")


genre_recall_A = genre_hits_A / (len(group_A_users) * k)
genre_recall_B = genre_hits_B / (len(group_B_users) * k)

print(f"Version A — GenreRecall@{k}: {genre_recall_A:.4f}")
print(f"Version B — GenreRecall@{k}: {genre_recall_B:.4f}")

ChatGPT was used to help with:
    - Graph plotting
    - Logging of results
    - Pytorch implmentation
    - Optuna set up
    - Implementation of implicit system